In [1]:
# 라이브러리 

import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm import tqdm
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [2]:
train=pd.read_csv('C:/Users/EunSeon/Downloads/p-sat-30th-winter-vacation-seminar/train.csv')
test=pd.read_csv('C:/Users/EunSeon/Downloads/p-sat-30th-winter-vacation-seminar/test.csv')

In [3]:
# 데이터 불러오기
ks_train_set = pd.read_csv('ks_train_set.csv')
ks_feature = pd.read_csv('ks_feature.csv')
ks_test_set = pd.read_csv('ks_test_set.csv')
test = pd.read_csv('test.csv')

In [5]:
sub = pd.DataFrame(test['ID_code'])   #추후 제출용을 위해

In [6]:
## (Modeling file과 동일한 과정)

# 클래스 불균형을 해소하기 위한 oversampling

oversample = RandomOverSampler(sampling_strategy=0.35,random_state= 42)
X_over, y_over = oversample.fit_resample(ks_feature, ks_train_set['target'])

# 샘플링 후 클래스 0과 클래스 1의 갯수 확인 

print(Counter(y_over))

# RandomSearchCV()를 통해 최적 파라미터 설정 후 모델 생성

clf = lgb.LGBMClassifier(bagging_fraction=0.85, bagging_freq=1, boost='gbdt',
               boosting_type='gbdt', class_weight=None, colsample_bytree=1,
               feature_fraction=1, gamma=1, importance_type='split',
               learning_rate=0.1, max_bin=256, max_depth=1,
               min_child_samples=153, min_child_weight=0.1, min_split_gain=0.0,
               n_estimators=4000, n_jobs=-1, num_leaves=4, num_threads=8,
               objective='binary', random_state=1, reg_alpha=0.1,
               reg_lambda=0, seed=500, silent=True, subsample=0.8,
               subsample_for_bin=200000, subsample_freq=0,boost_from_average = False)

Counter({0: 143922, 1: 50372})


In [7]:
# 모델 학습

clf.fit(X_over, y_over)

[LightGBM] [Warning] boosting is set with boosting_type=gbdt, will be overridden by boost=gbdt. Current value: boosting=gbdt
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] seed is set=500, random_state=1 will be ignored. Current value: seed=500
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.85, subsample=0.8 will be ignored. Current value: bagging_fraction=0.85
[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


LGBMClassifier(bagging_fraction=0.85, bagging_freq=1, boost='gbdt',
               boost_from_average=False, colsample_bytree=1, feature_fraction=1,
               gamma=1, max_bin=256, max_depth=1, min_child_samples=153,
               min_child_weight=0.1, n_estimators=4000, num_leaves=4,
               num_threads=8, objective='binary', random_state=1, reg_alpha=0.1,
               reg_lambda=0, seed=500, silent=True, subsample=0.8)

In [8]:
# 모델 적합 (test 데이터 적용)

pred = clf.predict(ks_test_set)

In [9]:
# 데이터 csv로 추출

sub['target'] = pd.DataFrame(pred)
sub.columns=['ID_code','target']
sub.to_csv('Submission_Team3.csv',index = False)

In [10]:
sub['target'].value_counts()

0    35838
1     4162
Name: target, dtype: int64